## Understanding and Predicting Property Maintenance Fines

This is the final assignment from Coursera course: Applied Machine Learning in Python.

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

def blight_model():
    import pandas as pd
    import numpy as np

    # Load data
    df_train = pd.read_csv('readonly/train.csv', encoding = "ISO-8859-1")
    df_test = pd.read_csv('readonly/test.csv')
    df_addresses = pd.read_csv('readonly/addresses.csv')
    df_latlons = pd.read_csv('readonly/latlons.csv')

    train = pd.merge(df_train, df_addresses,how = 'left', on = 'ticket_id')
    train = pd.merge(train, df_latlons, how ='left', on = 'address')

    test = pd.merge(df_test, df_addresses,how = 'left', on = 'ticket_id')
    test = pd.merge(test, df_latlons, how ='left', on = 'address')



    # Preprocessing
    # Examinations of data have been done and to keep the code concise, we will skip the code and only put comments here.

    # Drop all rows where 'complicance' is null from train data (test data don't include null 'compliance')
    train = train[-train['compliance'].isnull()]

    # Impute missing values of 'lat' and 'lon' in both train and test data.
    # Since 'lat' and 'lon' in test data contain NaN and we need to predict all the ticket_id in test data,
    # we can't simply drop all the NaN rows for 'lat' and 'lon'. Hence, use a easy way to impute missing values.
    train['lat'].fillna(method = 'ffill', inplace = True)
    train['lon'].fillna(method = 'ffill', inplace = True)
    test['lat'].fillna(method = 'ffill', inplace = True)
    test['lon'].fillna(method = 'ffill', inplace = True)

    # Remove features from train data
    train.drop(['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 
                'compliance_detail'], axis = 1, inplace = True) # Features only in train data

    train.drop(['violator_name', 'agency_name', 'inspector_name',
                'violation_street_number', 'violation_street_name', 'violation_zip_code', 'address',
                # Keep only 'lat' and 'lon' to reveal violation address information.
                # 'violation_zip_code' only contains NaN; 'address' comes from addresses data.
                'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code', 
                'non_us_str_code', 'country', 
                # For the mailing information, I drop all columns since violation information may already be good enough 
                # to point to the individual being ticketed.
                # 'country' only contains 'USA' in test data, hence, drop 'country' in the train data.
                'ticket_issued_date','hearing_date', 
                # These two dates are confusing because 'ticket_issued_date' doesn't always precceed hearing_date'.
                'violation_description',
                # There exists one-to-one match between'violation_description' and 'violation code'.
                'admin_fee', 'state_fee', 'late_fee', 'clean_up_cost', 'judgment_amount',
                # 'admin_fee', 'state_fee' and 'clean_up_cost' have only one value.'
                # late_fee' is 10% of 'fine_smount' and should not include since it will cause data leakage.
                # 'judgment_amount' = 'fine_amount'+'admin_fee'+'state_fee'+'late_fee', also should be dropped.
                # Therefore, only keep 'fine_amount' and 'discount_amount'.
                'grafitti_status'], 
                # 'grafitti_status' only contains NaN.
               axis = 1, inplace = True)

    # Select the same columns of train data from test data
    test = test[list(train.drop('compliance', axis = 1).columns)]


    # Convert some columns to string type
    train[['violation_code', 'disposition']] = train[['violation_code', 'disposition']].astype('str')
    test[['violation_code', 'disposition']] = test[['violation_code', 'disposition']].astype('str')

    # Convert string type columns to dummy variables
    train_dummies = pd.get_dummies(train, columns = ['violation_code', 'disposition'])
    test_dummies = pd.get_dummies(test, columns = ['violation_code', 'disposition'])

    # Find columns exist in both train and test data
    # After transforming the dummy variables, train and test data will have different columns. To simplify, I keep only 
    # columns in both data. These will cause information loss in both data. However, the result is still quite good.
    both_cols = [i for i in train_dummies.columns if i in test_dummies.columns]
    # In addition to the shared columns, train data should aslo contain the 'compliance' column.
    train_reduced = pd.concat([train_dummies[list(both_cols)],train_dummies['compliance']], axis = 1)
    test_reduced = test_dummies[both_cols]

    # Set ticket_id as the index
    train_reduced.set_index('ticket_id', inplace = True)
    test_reduced.set_index('ticket_id', inplace = True)



    # Machine learning
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score

    # Since this assignment doesn't provide the 'compliance' value of test data (I need to predict the probability of the compliance
    # for the test data to submit), I will split the train data into train test split to have a view of the model performance.
    X_train, X_test, y_train, y_test = train_test_split(train_reduced.drop(['compliance'], axis = 1), 
                                                        train_reduced['compliance'], random_state = 0)

    # As this assignment requires that the run time should generally be less than 10 mins, I won't use the GridSearchCV to tune the
    # parameters. Besides, the assignment doesn't allow the use of Neural Network.
    # I will use random forest here. As the Coursera autogreader has memory limit, I will be conservative when set 
    # hyperparameters. Even if I set only a few estimators, the result is still quite good.
    clf = RandomForestClassifier(n_estimators = 3, max_depth = 10, random_state = 0)
    clf.fit(X_train, y_train)

    print('Accuracy for test split: {}'.format(clf.score(X_test, y_test)))
    print('AUC for train split: {}'.format(roc_auc_score(y_train, clf.predict_proba(X_train)[:,1])))
    print('AUC for test split: {}'.format(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])))

    # Predict the probability that each corresponding ticket from readonly/test.csv will be paid
    test_proba = pd.Series(clf.predict_proba(test_reduced)[:,1], index = test_reduced.index, name = 'compliance')
    
    return test_proba   

In [2]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Accuracy for test split: 0.9331998999249437
AUC for train split: 0.7894287833960085
AUC for test split: 0.7713022011762874


ticket_id
284932    0.033377
285362    0.032397
285361    0.044133
285338    0.046230
285346    0.051985
285345    0.046230
285347    0.079654
285342    0.198456
285530    0.034497
284989    0.030666
285344    0.051985
285343    0.034497
285340    0.034497
285341    0.051985
285349    0.051985
285348    0.046230
284991    0.030666
285532    0.034497
285406    0.032397
285001    0.035716
285006    0.051985
285405    0.032397
285337    0.032397
285496    0.044133
285497    0.032397
285378    0.032397
285589    0.032397
285585    0.032397
285501    0.044133
285581    0.032397
            ...   
376367    0.044133
376366    0.034009
376362    0.254792
376363    0.209252
376365    0.044133
376364    0.034009
376228    0.034009
376265    0.034009
376286    0.296358
376320    0.034009
376314    0.025375
376327    0.304200
376385    0.364518
376435    0.209252
376370    0.272900
376434    0.044955
376459    0.061723
376478    0.044133
376473    0.034009
376484    0.115408
376482    0.044133
37

## Feedback from Coursera Autograder:
Your AUC of 0.769437151419 was awarded a value of 1.0 out of 1.0 total grades